In [1]:
import gym
from gym import spaces
import numpy as np

class TrafficGenerator(gym.Env):
    
    #doesnt really workj
    def getObs(self):
        # Loop over the packet types
        for i in range(len(self.mean_delay_req)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            print(np.average(current_queue), current_length, current_waiting_time )
            
            
           
            observation.append([current_length, current_waiting_time])

       
        observation = np.array(observation)
        return observation
    
    def __init__(self):
        
        self.action_space = spaces.Discrete(3)
        
        # Define the observation space (number of packets in each queue and their waiting time)
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(3,2), dtype=int)
        
        # pancket info (DataType, arrival_rate, mean_delay )
        #self.packetInfo = [[0, 0.3, 6],[1, 0.25, 4],[2, 0.4, float('inf')]]
        
        self.dataType = [0, 1, 2]
        self.arrival_rate = [0.4, 0.4, 0.4]
        self.mean_delay_req = [6, 4, float('inf')]
        self.packetInfo = [[elem1, elem2, elem3] for elem1, elem2, elem3 in zip(self.dataType, self.arrival_rate, self.mean_delay_req)]
        self.curr_mean_delay_best_effort = 0
        self.packet = 1
        self.timeslot = 1
        
        # Initialize the queues
        self.queues = [[], [], []]
      
    
    def step(self, action):  
        #packet generator
        for i in range(len(self.packetInfo)):
            for sublist in self.packetInfo:
                if sublist[0] == i and np.random.uniform() < sublist[1]:
                    self.queues[i].append(self.packet)
                    print("Appending to queue", i)
        
        for i in range(len(self.queues)):
            for j in range(len(self.queues[i])):
                self.queues[i][j] -= self.timeslot
                
        
        # services the queues
        for i in range(len(self.queues)):
            print("queue ", i, ": has a length of", len(self.queues[i]))
        print("ACTION queue ", action, ": has a length of", len(self.queues[action]))
        if action < 3 and len(self.queues[action]) > 0:
            self.queues[action].pop(0)
            for i in range(len(self.queues[action])):
                self.queues[action][i] -= 1
        

        
        #TODO rewardoptions?
        

        
         # Loop over the packet types
        observation = []
        for i in range(len(self.mean_delay_req)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            
    
            observation.append([current_length, current_waiting_time])
            if i == 2:
                #reward option
                if self.curr_mean_delay_best_effort == 0:
                    curr_mean_delay_best_effort = current_waiting_time
                if self.curr_mean_delay_best_effort < current_waiting_time:
                    self.curr_mean_delay_best_effort = current_waiting_time
                    reward = 1
                elif self.curr_mean_delay_best_effort > current_waiting_time:
                    self.curr_mean_delay_best_effort = current_waiting_time
                    reward = -1
                else:
                    reward = 1
                
                print(reward)
                print(self.curr_mean_delay_best_effort)
                print(current_waiting_time)

        observation = np.array(observation)
        
        done = len(self.queues[0]) + len(self.queues[1]) + len(self.queues[2]) == 0
        return observation, done, reward, {}
        
    def reset(self):
        # Reset the queues
        self.queues = [[], [], []]
        return np.array([[len(self.queues[i]), 0.0] for i in range(len(self.mean_delay_req))])
        
    def render(self):
        pass

In [2]:
env = TrafficGenerator()
obs = env.reset()
done = False
x = 0
while x < 100:
    print("~~~~~~~~~")
    print(obs[:, 0])
    
    if obs[:, 0][0] > 0 or obs[:, 0][1] > 0:
        action = np.argmax(obs[:2, 0])
    else:
        action = 2 
    
    print("chosen", action)
    obs, done, reward, _ = env.step(action)
    print(obs)
    print(reward)
    x = x + 1

~~~~~~~~~
[0. 0. 0.]
chosen 2
Appending to queue 1
queue  0 : has a length of 0
queue  1 : has a length of 1
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
1
0
0.0
[[0. 0.]
 [1. 0.]
 [0. 0.]]
1
~~~~~~~~~
[0. 1. 0.]
chosen 1
Appending to queue 1
queue  0 : has a length of 0
queue  1 : has a length of 2
queue  2 : has a length of 0
ACTION queue  1 : has a length of 2
1
0
0.0
[[ 0.  0.]
 [ 1. -1.]
 [ 0.  0.]]
1
~~~~~~~~~
[0. 1. 0.]
chosen 1
Appending to queue 2
queue  0 : has a length of 0
queue  1 : has a length of 1
queue  2 : has a length of 1
ACTION queue  1 : has a length of 1
1
0
0.0
[[0. 0.]
 [0. 0.]
 [1. 0.]]
1
~~~~~~~~~
[0. 0. 1.]
chosen 2
Appending to queue 0
Appending to queue 1
Appending to queue 2
queue  0 : has a length of 1
queue  1 : has a length of 1
queue  2 : has a length of 2
ACTION queue  2 : has a length of 2
-1
-1.0
-1.0
[[ 1.  0.]
 [ 1.  0.]
 [ 1. -1.]]
-1
~~~~~~~~~
[1. 1. 1.]
chosen 0
Appending to queue 0
Appending to queue 1
queue  0 : has a len

In [3]:
# import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# # Define the Q-learning agent
# class QLearningAgent:
#     def __init__(self, observation_space, action_space, learning_rate=0.1, discount_factor=0.9):
#         self.observation_space = observation_space
#         self.action_space = action_space
#         self.learning_rate = learning_rate
#         self.discount_factor = discount_factor
#         self.q_table = np.zeros((self.observation_space[0], self.observation_space[1], self.action_space))

#     def choose_action(self, state):
#         state = tuple(state.astype(int))  # Convert the state to a tuple of integers
#         return np.argmax(self.q_table[state])

#     def update_q_table(self, state, action, reward, next_state, done):
#         state = tuple(state.astype(int))  # Convert the state to a tuple of integers
#         next_state = tuple(next_state.astype(int))  # Convert the next state to a tuple of integers
#         q_value = self.q_table[state][action.item()]  # Access the Q-value using action.item()
#         if done:
#             q_value += self.learning_rate * (reward - q_value)
#         else:
#             max_q_value = np.max(self.q_table[next_state])
#             q_value += self.learning_rate * (reward + self.discount_factor * max_q_value - q_value)
#         self.q_table[state][action.item()] = q_value

# # Define the environment and observation space
# observation_space = (3, 2)
# action_space = 3

# # Create an instance of the Q-learning agent
# agent = QLearningAgent(observation_space, action_space)

# # Create the Q-learning model using Keras
# model = Sequential()
# model.add(Dense(64, input_shape=observation_space, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(action_space, activation='linear'))
# model.compile(loss='mse', optimizer='adam')

# num_episodes = 1000
# # Train the Q-learning model
# for episode in range(num_episodes):
#     state = env.reset()
#     done = False
#     while not done:
#         action = agent.choose_action(state)
#         next_state, reward, done, _ = env.step(action)
#         print(state)
#         agent.update_q_table(state, action, reward, next_state, done)
#         state = next_state

# # Save the trained Q-learning model
# model.save('q_learning_model.h5')



In [6]:
# q_table = np.zeros(env.observation_space)
print(env.observation_space)


Box(0, 2147483645, (3, 2), int32)
